# 02 — Simulator & Metrics

**Objectif :** charger une instance, représenter un planning, le **simuler**, puis calculer
les métriques d’optimisation :
- makespan $C_{max}$
- tardiness \($T_j\$), somme des retards
- score multi-critères


## 1) Principe de la simulation

Un planning est donné par des séquences \(\pi_k\) (une par machine).  
La simulation reconstruit les temps de début et de fin :

- $(S_j)$ : start time de la tâche $(j)$
- $(C_j)$ : completion time de la tâche $(j)$

Sur une machine $(k)$ exécutant $pi_k=(j_1,j_2,\dots)$ :

- $S_{j_1}$ = $max(0, r_{j_1}))$
- $S_{j_t}$ = $max(C_{j_{t-1}}, r_{j_t})$ pour $(t$ge2$)$
- $(C_{j_t} = S_{j_t} + p_{j_t,k}$)

Ensuite, on calcule les métriques globales.


In [1]:
import json
import os
import numpy as np
import sys, pathlib

# Suppose que le notebook est dans montecarlo-scheduling/notebooks/
PROJECT_ROOT = pathlib.Path.cwd().resolve().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

INSTANCES_DIR = PROJECT_ROOT / "data" / "instances"
print("INSTANCES_DIR:", INSTANCES_DIR)


INSTANCES_DIR: /data/instances


## 2) Chargement d’une instance

Les instances ont été générées au format JSON dans `data/instances/`.


In [3]:
def load_instance(path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

# Fix: INSTANCES_DIR was incorrectly set, leading to FileNotFoundError.
# Redefine INSTANCES_DIR to point to the correct path in Colab.
INSTANCES_DIR = pathlib.Path("/content") / "data" / "instances"

# Exemple : adapter le nom si besoin
instance_path = INSTANCES_DIR / "instance_50jobs_5machines.json"
instance = load_instance(instance_path)

n_jobs = instance["n_jobs"]
n_machines = instance["n_machines"]
p = instance["processing_times"]
d = instance.get("deadlines", None)
w = instance.get("weights", None)

print("Loaded:", instance["name"])
print("n_jobs:", n_jobs, "| n_machines:", n_machines)
print("deadlines:", d is not None, "| weights:", w is not None)


Loaded: instance_50jobs_5machines
n_jobs: 50 | n_machines: 5
deadlines: True | weights: False


## 3) Représentation d’une solution (planning)

On représente un planning comme une liste de listes :

- `solution[k]` = liste ordonnée des tâches exécutées sur la machine \(k\)

Exemple (2 machines) :
- machine 0 : [0, 3, 1]
- machine 1 : [2, 4]


In [4]:
def is_valid_solution(solution, n_jobs):
    flat = [job for seq in solution for job in seq]
    return (len(flat) == n_jobs) and (sorted(flat) == list(range(n_jobs)))


## 4) Solution de test

On crée une solution simple pour tester le simulateur (sans optimisation).


In [5]:
# Solution "round-robin": assigner les jobs dans l'ordre aux machines
solution = [[] for _ in range(n_machines)]
for j in range(n_jobs):
    solution[j % n_machines].append(j)

print("Valid solution?", is_valid_solution(solution, n_jobs))
print("Example machine 0:", solution[0][:10], "...")


Valid solution? True
Example machine 0: [0, 5, 10, 15, 20, 25, 30, 35, 40, 45] ...


## 5) Simulation

La simulation calcule :
- `start[j]` = temps de début de la tâche \(j\)
- `end[j]` = temps de fin de la tâche \(j\)
- `machine_time[k]` = temps final de la machine \(k\)
- `Cmax` = makespan


In [6]:
def simulate(instance, solution):
    n_jobs = instance["n_jobs"]
    n_machines = instance["n_machines"]
    p = instance["processing_times"]
    releases = instance.get("releases", None)
    if releases is None:
        releases = [0.0] * n_jobs

    start = [0.0] * n_jobs
    end = [0.0] * n_jobs
    machine_time = [0.0] * n_machines

    for k in range(n_machines):
        t = 0.0
        for j in solution[k]:
            t = max(t, float(releases[j]))
            start[j] = t
            t = t + float(p[j][k])
            end[j] = t
        machine_time[k] = t

    Cmax = max(end) if end else 0.0
    return {"start": start, "end": end, "machine_time": machine_time, "Cmax": Cmax}


In [7]:
sim = simulate(instance, solution)
print("Cmax:", sim["Cmax"])
print("Machine times (first 5):", sim["machine_time"][:5])


Cmax: 119.0
Machine times (first 5): [113.0, 96.0, 102.0, 86.0, 119.0]


## 6) Calcul des métriques

On calcule :
- makespan $(C_{\max}$)
- tardiness $(T_j = max(0, C_j - d_j)$) si deadlines
- somme des retards $(sum_j T_j$)
- score : $(alpha C_{max} + beta sum_j T_j + gamma sum_j w_j T_j$)


In [8]:
def compute_metrics(instance, sim, alpha=1.0, beta=0.2, gamma=0.0):
    end = sim["end"]
    Cmax = float(sim["Cmax"])

    deadlines = instance.get("deadlines", None)
    weights = instance.get("weights", None)

    tardiness = [0.0] * instance["n_jobs"]
    weighted_tardiness = [0.0] * instance["n_jobs"]

    sumT = 0.0
    sumWT = 0.0

    if deadlines is not None:
        for j in range(instance["n_jobs"]):
            tj = max(0.0, float(end[j]) - float(deadlines[j]))
            tardiness[j] = tj
            sumT += tj

            if weights is not None:
                wtj = float(weights[j]) * tj
                weighted_tardiness[j] = wtj
                sumWT += wtj

    score = alpha * Cmax + beta * sumT + gamma * sumWT

    return {
        "Cmax": Cmax,
        "tardiness": tardiness,
        "weighted_tardiness": weighted_tardiness,
        "sumT": sumT,
        "sumWT": sumWT,
        "score": score
    }


In [9]:
metrics = compute_metrics(instance, sim, alpha=1.0, beta=0.2, gamma=0.0)
print("Cmax:", metrics["Cmax"])
print("sumT:", metrics["sumT"])
print("score:", metrics["score"])


Cmax: 119.0
sumT: 213.67759489035114
score: 161.73551897807022


## 7) Vérifications rapides

On vérifie :
- la solution est valide
- les temps de fin sont cohérents
- le score est bien défini

Ces checks évitent de propager des erreurs dans les algorithmes Monte Carlo.


In [10]:
assert is_valid_solution(solution, n_jobs), "Solution invalide"
assert all(sim["end"][j] >= sim["start"][j] for j in range(n_jobs)), "Incohérence start/end"
assert metrics["score"] >= 0, "Score négatif inattendu (vérifier paramètres/données)"
print("Sanity checks OK ✅")


Sanity checks OK ✅


## 8) Conclusion

Nous avons :
- chargé une instance,
- défini une représentation de solution,
- simulé un planning,
- calculé les métriques et un score multi-critères.

Prochaine étape : **03_Baselines_Heuristics.ipynb**
- implémenter des baselines (random, heuristiques simples)
- comparer leurs performances sur plusieurs seeds
